In [29]:
from data import *
from utils.augmentations import SSDAugmentation
import torch
from torch.autograd import Variable
from box_utils import match, point_form, jaccard, match, intersect
from SSD_generate_anchors import generate_ssd_priors

In [2]:
from pathlib import Path

In [3]:
root = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'
cfg = voc

In [4]:
dataset = VOCDetection(root = root,
                      transform=SSDAugmentation(cfg['min_dim'], MEANS))

In [5]:
len(dataset)

16551

In [6]:
sample = dataset[12706]
print(sample[0].size())
print(sample[1].shape)

torch.Size([3, 300, 300])
(5, 5)


In [7]:
import torch.utils.data as data

In [8]:
data_loader = data.DataLoader(dataset, 4,
                                  num_workers=2,
                                  shuffle=True,
                                  collate_fn=detection_collate,
                                  pin_memory=True)
batch_iterator = iter(data_loader)

# for i, d in enumerate(batch_iterator):
#     print(d[0])
#     print(d[1])
#     if i ==1:
#         break

In [43]:
import collections

SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

priors = generate_ssd_priors(specs)

In [44]:
priors = torch.from_numpy(priors)
priors = priors.float()

In [45]:
priors_point_form = point_form(priors)
print(priors_point_form.size())

torch.Size([8732, 4])


In [47]:
default = priors_point_form.data

In [48]:
print(priors.size())

torch.Size([8732, 4])


In [56]:
for idx, (image, target) in enumerate(batch_iterator):
#     print(image.size(0))
    bs = image.size(0)
    for i in range(bs):
        label = target[i][:,:-1].data
#         print(label.size())
        inter = intersect(label, default)
        union = jaccard(label, default)
        print(inter.size())
        print(union.size())
#         print(overlap)
    if idx == 0:
        break

torch.Size([1, 8732])
torch.Size([1, 8732])
torch.Size([2, 8732])
torch.Size([2, 8732])
torch.Size([4, 8732])
torch.Size([4, 8732])
torch.Size([2, 8732])
torch.Size([2, 8732])


In [63]:
union.max(1, keepdim=True)


torch.return_types.max(
values=tensor([[0.7280],
        [0.6204]]),
indices=tensor([[8332],
        [7511]]))

In [64]:
union.max(0, keepdim=True)

torch.return_types.max(
values=tensor([[0.0000, 0.0000, 0.0000,  ..., 0.1509, 0.1596, 0.2241]]),
indices=tensor([[1, 1, 1,  ..., 0, 0, 0]]))

In [10]:
from layers.modules import MultiBoxLoss
from ssd import build_ssd

In [11]:
cfg = voc

In [12]:
cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [42]:
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net.cuda()

In [44]:
for i, d in enumerate(batch_iterator):
    image = d[0].cuda()
    out = net(image)
    if i == 0:
        break

In [45]:
print(out[0].size())
print(out[1].size())
print(out[2].size())


torch.Size([4, 8732, 4])
torch.Size([4, 8732, 21])
torch.Size([8732, 4])


In [50]:
criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                             False, True)

/home/vlsilab/anaconda2/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """Entry point for launching an IPython kernel.


In [51]:
# load train data
images, targets = next(batch_iterator)

In [52]:
images = Variable(images.cuda())
targets = [Variable(ann.cuda(), volatile=True) for ann in targets]

/home/vlsilab/anaconda2/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [53]:
out = net(images)

In [55]:
# criterion(out, targets)